<a href="https://colab.research.google.com/github/Shumail-Khan/Helmet-Detection/blob/main/Helmet_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Motorcycle Helmet Detection with YOLOv8

**Setup required:**
1. Go to the left sidebar → 🔑 Secrets tab
2. Add a new secret:
   - Name: `ROBOFLOW_API_KEY`
   - Value: your Roboflow API key (from https://app.roboflow.com/settings/api)
   - Toggle "Notebook access" ON

In [ ]:
!pip install ultralytics roboflow

# Dataset Preparation with Roboflow

Steps:

<ol>
  <li>Go to Roboflow → Sign up / log in (free).</li>
  <li>Go to similar datasets → Click Fork (or "Use this Dataset").</li>
  <li>Add own images:</li>
  <ul>
    <li>Search Google/Images for "motorcycle rider helmet" and "motorcycle rider no helmet".</li>
    <li>Upload them → Use the Annotate tool to draw boxes and label.</li>
  </ul>

  <li>Generate a new Version:</li>
    <ul>
    <li>Preprocessing: Auto-Orient + Resize (Stretch to 640×640)</li>
    <li>Augmentations: Flip, Rotation (±15°), Blur</li>
    </ul>

  <li>Export the dataset:</li>
  <ul>
    <li>Format → YOLOv8</li>
    <li>Download the zip (or copy the code snippet)</li>
    <li>Unzip it → you will get a folder with data.yaml, train/, valid/, test/ folders.</li>
  </ul>

</ol>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
from google.colab import userdata
rf = Roboflow(api_key=userdata.get('ROBOFLOW_API_KEY'))
project = rf.workspace("shumail-39pbo").project("bike-helmet-detection-2vdjo-byw6k")
version = project.version(2)
dataset = version.download("yolov8")

# Train YOLOv8

In [ ]:
from ultralytics import YOLO

# Load a pretrained model (nano = fast & good enough for start)
model = YOLO("yolov8n.pt")

# Train
results = model.train(
    data="./Bike-Helmet-Detection-2/data.yaml",
    epochs=50,
    imgsz=640,
    batch=8
)

<bold><em>After training, best model is saved at runs/detect/train/weights/best.pt</bold></em>

# Fine-tuning & mAP Evaluation

<ul>
  <li>Look at the training logs</li>
  <li>Common improvements:</li>
  <ul>
    <li>Increase epochs to 100–150 if mAP@50 is still rising.</li>
    <li>Try yolov8s.pt (small) instead of nano for better accuracy.</li>
    <li>Add more augmentations in Roboflow (mosaic, hsv, etc.) and re-export, to create new versions of datasets.</li>
  </ul>
</ul>

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")
metrics = model.val()   # runs on validation set automatically
print(metrics.box.map)  # mAP@50:95
print(metrics.box.map50)  # mAP@50 (most common metric)

# Inference Script + Webcam Demo

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import os
from IPython.display import Image, display

# Change this path if your best.pt is somewhere else
model = YOLO("runs/detect/train/weights/best.pt")

In [ ]:
test_folder = "Bike-Helmet-Detection-2/test/images"   # ← your test images folder

# Create a folder to save results
os.makedirs("test_results", exist_ok=True)

for img_name in os.listdir(test_folder):
    if img_name.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(test_folder, img_name)

        results = model(img_path, conf=0.5, iou=0.45)   # adjust conf/iou if needed

        # Show in Colab
        annotated = results[0].plot()
        cv2_imshow(annotated)

        # Save annotated image
        save_path = f"test_results/{img_name}"
        cv2.imwrite(save_path, annotated)
        print(f"✓ Processed and saved: {img_name}")

print("All test images processed! Check the 'test_results' folder.")

In [ ]:
from google.colab import files

# 1. Upload your test video
uploaded = files.upload()
video_path = list(uploaded.keys())[0]   # e.g. "test_video.mp4"

# 2. Output video path
output_path = "output_helmet_detection.mp4"

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 20.0,
                      (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=0.5)
    annotated_frame = results[0].plot()

    out.write(annotated_frame)

    # Show every 10th frame in Colab so you can see progress
    if frame_count % 10 == 0:
        cv2_imshow(annotated_frame)
        print(f"Processing frame {frame_count}")

    frame_count += 1

cap.release()
out.release()
print("Video processing complete! Download below:")
files.download(output_path)

In [ ]:
# Video or webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    results = model(frame, conf=0.5)
    annotated = results[0].plot()
    cv2.imshow("Helmet Detection", annotated)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

# Report Accuracy & False Positives

Dataset size & splits
<ul>
  <li>Total Images: 4587</li>
  <li>Train Set: 3876</li>
  <li>Valid Set: 476</li>
  <li>Test Set: 235</li>
</ul>
Final mAP@50, Precision, Recall
  <ul>
    <li>mAP@50:95 = 0.62</li>
    <li>mAP@50 = 0.88</li>
  </ul>
Confusion matrix (from model.val())<br>
<img src='.\runs\detect\val\confusion_matrix_normalized.png' alt="Not Available">
False positives discussion:
<ul>
  <li>FPs: dark helmets at night, hats mistaken for helmets, people with round heads.</li>
  <li>How to reduce: higher conf threshold (0.6–0.7), more night-time images in dataset, or add a "person" class first then check helmet on head.</li>
</ul>